# Pay Weekly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import psycopg2
import pandas as pd
import matplotlib
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

weekly total before running time.

In [ ]:
%%sql stat_premium_accounts_completed  <<
SELECT count(*) FILTER (WHERE payment_method = 'PAD') AS PAD
,count(*) FILTER (WHERE payment_method = 'DRAWDOWN') AS BCOL
FROM payment_accounts

Save to CSV

In [ ]:
filename = os.getenv('DATA_DIR', '')+'pay_weekly_stats_till_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'

with open(filename, 'w') as f:      
    f.write('Total Number of PAD and BCOL Users (overall):\n')
    stat_premium_accounts_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_nro_weekly_completed  <<
SELECT count(*)
FROM invoices i
WHERE
i.corp_type_code = 'NRO'
AND i.payment_method_code IN ('PAD','DRAWDOWN')
AND i.invoice_status_code IN ('PAID', 'APPROVED')
AND date(i.created_on at time zone 'utc' at time zone 'pst')  > date(current_date - 1 - interval '1 weeks')
AND date(i.created_on at time zone 'utc' at time zone 'pst')  <= date(current_date - 1);

In [ ]:
with open(filename, 'a') as f:      
    f.write('\n\n Weekly Number of NR Submitted in Premium Account:\n')
    stat_nro_weekly_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_nro_total_completed  <<
SELECT count(*)
FROM invoices i
WHERE
i.corp_type_code = 'NRO'
AND i.payment_method_code IN ('PAD','DRAWDOWN')
AND i.invoice_status_code IN ('PAID', 'APPROVED');

In [ ]:
with open(filename, 'a') as f:      
    f.write('\n\n Total Number of NR Submitted in Premium Account:\n')
    stat_nro_total_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)